In [1]:
import xarray as xr
import rioxarray as rxr
from rioxarray.merge import merge_arrays
from rasterio.enums import Resampling
import geopandas as gpd
import os

# Creates a 150m DEM for the UCRB

In [2]:
path_str = '/storage/dlhogan/sos/data/dem/'
filenames = os.listdir(path_str)
dem_list=[]
for filename in filenames:
    print(f'Working on {filename}...')
    path_to_file = os.path.join(path_str,filename)
    raster = rxr.open_rasterio(path_to_file)
    downscale_factor = 5
    
    # Caluculate new height and width using upscale_factor
    new_width = raster.rio.width / downscale_factor
    new_height = raster.rio.height / downscale_factor
    
    #downsample raster to 150 m from 30 m
    down_sampled = raster.rio.reproject(raster.rio.crs, shape=(int(new_height), int(new_width)), resampling=Resampling.bilinear)
    dem_list.append(down_sampled)
    print('Done!')

Working on USGS_1_n36w106_20220801.tif...
Done!
Working on USGS_1_n36w107_20220801.tif...
Done!
Working on USGS_1_n36w108_20220801.tif...
Done!
Working on USGS_1_n36w112_20230418.tif...
Done!
Working on USGS_1_n37w106_20220801.tif...
Done!
Working on USGS_1_n37w107_20220801.tif...
Done!
Working on USGS_1_n37w108_20220801.tif...
Done!
Working on USGS_1_n37w109_20220720.tif...
Done!
Working on USGS_1_n36w109_20211207.tif...
Done!
Working on USGS_1_n37w110_20220720.tif...
Done!
Working on USGS_1_n37w111_20211215.tif...
Done!
Working on USGS_1_n37w112_20230418.tif...
Done!
Working on USGS_1_n38w106_20211208.tif...
Done!
Working on USGS_1_n38w107_20220720.tif...
Done!
Working on USGS_1_n38w108_20220720.tif...
Done!
Working on USGS_1_n36w110_20211207.tif...
Done!
Working on USGS_1_n36w111_20220303.tif...
Done!
Working on USGS_1_n38w111_20211215.tif...
Done!
Working on USGS_1_n38w112_20211215.tif...
Done!
Working on USGS_1_n39w106_20211208.tif...
Done!
Working on USGS_1_n39w107_20220331.tif..

In [3]:
co_crs = 32613
# Merge, reproject and mask array
merged_dem = merge_arrays(dem_list).squeeze()
merged_dem = merged_dem.rio.reproject(co_crs)
merged_dem = merged_dem.where(merged_dem>0)
# Load in UCRB boundary
ucrb_boundary = gpd.read_file('../basin_characteristics/Upper_Colorado_River_Basin_Boundary.json')
ucrb_boundary = ucrb_boundary.to_crs(co_crs)
# Clip the dem to the UCRB
merged_dem_ucrb = merged_dem.rio.clip(ucrb_boundary.geometry)
merged_dem_ucrb_ma = merged_dem_ucrb.where(merged_dem_ucrb>0)

In [4]:
merged_dem_ucrb_ma.rio.to_raster('../basin_characteristics/ucrb_dem_32613_150m.tif')

### Clip Tree Cover Raster to UCRB Extent

In [2]:
from shapely.geometry import mapping, Polygon
tree_cover = rxr.open_rasterio('../basin_characteristics/ucrb_tree_cover_fraction.tif')
ucrb_boundary = gpd.read_file('../basin_characteristics/Upper_Colorado_River_Basin_Boundary.json')
ucrb_boundary = ucrb_boundary.to_crs(tree_cover.rio.crs)
ucrb_tree_cover = tree_cover.rio.clip(ucrb_boundary.geometry.apply(mapping), drop=False, invert=False, from_disk=True)


In [3]:
ucrb_tree_cover.rio.to_raster('../basin_characteristics/ucrb_tree_cover_fraction_cleaned.tif')